In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#モジュールの追加
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

#パラメーターチューニング用
#from optuna.integration import lightgbm as lgb

In [ ]:
#データの読み込み
df_test = pd.read_csv('/kaggle/input/test-data/test.csv')
df_train = pd.read_csv('/kaggle/input/train-data/train.csv')

In [ ]:
#訓練データとテストデータをマージ
df_train['DataType'] = 'train'
df_test['DataType'] = 'test'
df_test['revenue'] = 0
all_data = pd.concat([df_train,df_test],axis=0).reset_index(drop=True)

#6年前を想定して営業日数を特徴量に加える
all_data["Open Date"] = pd.to_datetime(all_data["Open Date"])#日付データ型に変換
all_data["year"] = all_data["Open Date"].apply(lambda x:x.year)
all_data["month"] = all_data["Open Date"].apply(lambda x:x.month)
all_data["day"] = all_data["Open Date"].apply(lambda x:x.day)
all_data["reference"] = "2015-07-09"
all_data["reference"] = pd.to_datetime(all_data["reference"])
all_data["period"] = (all_data["reference"] - all_data["Open Date"]).apply(lambda x: x.days)
#ノイズの除去
all_data = all_data.drop('Open Date', axis=1)
all_data = all_data.drop('reference', axis=1)

In [ ]:
# 訓練データ特徴量をリストに変換
cat_col = all_data.dtypes[all_data.dtypes=='object'].index.tolist()
num_col = all_data.dtypes[all_data.dtypes!='object'].index.tolist()

other_col = ['Id','DataType']
# 余計な要素をリストから削除
cat_col.remove('DataType') #学習データ・テストデータ区別フラグ除去
num_col.remove('Id') #Id削除

In [ ]:
#カテゴリカル変数をダミー化
cat = pd.get_dummies(all_data[cat_col])

# データの統合
all_data = pd.concat([all_data[other_col],all_data[num_col].fillna(0),cat],axis=1)


In [ ]:
#訓練データとテストデータを抽出
train_ = all_data[all_data['DataType']=='train'].drop(['DataType','Id'], axis=1).reset_index(drop=True)
test_ = all_data[all_data['DataType']=='test'].drop(['DataType','revenue'], axis=1).reset_index(drop=True)
x_ = train_.drop('revenue',axis=1)#revenue列の以外
y_1 = train_.loc[:, ['revenue']]#revenue列
y_ = np.log(y_1)
#7:3の比率で訓練データとテストデータを分割
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3,random_state=10)
# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
test_feature = test_.drop('Id',axis=1)

In [ ]:
# LightGBMパラメータ
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : {'l1'},
        'verbosity': -1,
        'feature_pre_filter': False,
        'num_leaves' : 31,
        'learning_rate' : 0.1,
        'feature_fraction' : 0.5,
        'bagging_fraction' : 0.5,
        'bagging_freq': 2,
        'min_child_samples': 20,
        'verbose' : 0,
}

In [ ]:
# LightGBM学習
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=100,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

# LightGBM推論
prediction_lgb = np.exp(gbm.predict(test_feature))

In [ ]:
# Idを取得
Id = np.array(df_test["Id"]).astype(int)
# 予測データとIdをデータフレームへ落とし込む
result = pd.DataFrame(prediction_lgb, Id,columns = ["prediction"])
# csvとして書き出し
result.to_csv("prediction_revenu.csv", index_label = ["Id"])


In [ ]:
result